In [1]:
% matplotlib inline

import sys
import numpy as np
import scipy.io as sio
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from mpl_toolkits.basemap import Basemap
from datetime import date
import pandas as pd
from netCDF4 import Dataset

/Users/sclayton/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:7: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "/Users/sclayton/anaconda/lib/python2.7/runpy.py", line 162, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/Users/sclayton/anaconda/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/Users/sclayton/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/Users/sclayton/anaconda/lib/python2.7/site-packages/traitlets/config/application.py", line 592, in launch_instance
    app.start()
  File "/Users/sclayton/anaconda/lib/python2.7/site-packages/ipykernel/kernelap

In [2]:
def daynToDate(dt):
    year = dt / 1000
    dayn = dt % 1000
    dat = date.fromordinal(date(year, 1, 1).toordinal() + dayn - 1)
    return dat


def loadLCS(path):
    content = sio.loadmat(path)
    X = content['X1']
    Y = content['Y1']
    lon = content['lon']
    lat = content['lat']
#     X = X - 360
    return X, Y, lon, lat


# def tracerBlock(X, Y, lonIndex1, lonIndex2, latIndex1, latIndex2):
#     lons = X[lonIndex1:lonIndex2+1, latIndex1:latIndex2+1]
#     lats = Y[lonIndex1:lonIndex2+1, latIndex1:latIndex2+1]
#     lons = lons.flatten()
#     lats = lats.flatten()
#     return lons, lats   

def tracerBlock(X, Y, lon_index, lat_index, hw):
    lons = []
    lats = []
    for on in lon_index:
        for at in lat_index:
            lons.append((X[on-hw:on+hw+1, at-hw:at+hw+1]).flatten())
            lats.append((Y[on-hw:on+hw+1, at-hw:at+hw+1]).flatten())
    lons = np.asarray(lons).flatten()
    lats = np.asarray(lats).flatten()
    return lons, lats   
                


def bwFat():
    ################ zonal elongation ######################
    h, w = 50, 200
    lonsPolar, latsPolar = tracerBlock(X, Y, 4950, 4950+w, 3420, 3420+h)    
    lonsNorth, latsNorth = tracerBlock(X, Y, 4950, 4950+w, 3200, 3200+h)    
    #lonsTZ, latsTZ = tracerBlock(X, Y, 4950, 4950+w, 3025, 3025+h)
    lonsTZ1, latsTZ1 = tracerBlock(X, Y, 4950, 4950+w, 3000, 3030)
    lonsTZ2, latsTZ2 = tracerBlock(X, Y, 4950, 4950+w, 3030, 3060)
    lonsTZ3, latsTZ3 = tracerBlock(X, Y, 4950, 4950+w, 3060, 3090)
    lonsTZ4, latsTZ4 = tracerBlock(X, Y, 4950, 4950+w, 3120, 3150)
    lonsSouth, latsSouth = tracerBlock(X, Y, 4950, 4950+w, 2840, 2840+h)
    ########################################################
    return lonsPolar, latsPolar, lonsNorth, latsNorth, lonsTZ1, latsTZ1, lonsTZ2, latsTZ2, lonsTZ3, latsTZ3, lonsTZ4, latsTZ4, lonsSouth, latsSouth

    

def bwSkinny():
    ################ meridional elongation ######################
    h, w = 50, 30
    lonsPolar, latsPolar = tracerBlock(X, Y, 5032, 5032+w, 3420, 3420+h)    
    lonsNorth, latsNorth = tracerBlock(X, Y, 5032, 5032+w, 3200, 3200+h)    
    #lonsTZ, latsTZ = tracerBlock(X, Y, 5032, 5032+w, 3025, 3025+h)
    lonsTZ1, latsTZ1 = tracerBlock(X, Y, 5032, 5032+w, 2990, 3030)
    lonsTZ2, latsTZ2 = tracerBlock(X, Y, 5032, 5032+w, 3030, 3060)
    lonsTZ3, latsTZ3 = tracerBlock(X, Y, 5032, 5032+w, 3060, 3100)
    lonsTZ4, latsTZ4 = tracerBlock(X, Y, 5032, 5032+w, 3120, 3150)
    lonsSouth, latsSouth = tracerBlock(X, Y, 5032, 5032+w, 2840, 2840+h)
    ########################################################
    return lonsPolar, latsPolar, lonsNorth, latsNorth, lonsTZ1, latsTZ1, lonsTZ2, latsTZ2, lonsTZ3, latsTZ3, lonsTZ4, latsTZ4, lonsSouth, latsSouth



def grad2Domain():
    lonsPolar, latsPolar, lonsNorth, latsNorth, lonsTZ1, latsTZ1, lonsTZ2, latsTZ2, lonsTZ3, latsTZ3, lonsTZ4, latsTZ4, lonsSouth, latsSouth = bwFat()
    #lonsPolar, latsPolar, lonsNorth, latsNorth, lonsTZ1, latsTZ1, lonsTZ2, latsTZ2, lonsTZ3, latsTZ3, lonsTZ4, latsTZ4, lonsSouth, latsSouth = bwSkinny()
    
    map = Basemap(llcrnrlon=-168,llcrnrlat=18.,urcrnrlon=-140,urcrnrlat=55,\
        rsphere=(6378137.00,6356752.3142),\
        resolution='l',area_thresh=1000.,projection='lcc',\
        lat_1=40.,lon_0=-158.)
    map.plot(lonsPolar, latsPolar, 'o', color='maroon', markeredgecolor='none', markersize=1, latlon=True)
    map.plot(lonsNorth, latsNorth, 'o', color='springgreen', markeredgecolor='none', markersize=1, latlon=True)
    map.plot(lonsTZ1, latsTZ1, 'o', color='crimson', markeredgecolor='none', markersize=1, latlon=True)
    map.plot(lonsTZ2, latsTZ2, 'o', color='white', markeredgecolor='none', markersize=1, latlon=True)
    map.plot(lonsTZ3, latsTZ3, 'o', color='darkslategrey', markeredgecolor='none', markersize=1, latlon=True)
    map.plot(lonsTZ4, latsTZ4, 'o', color='darkorange', markeredgecolor='none', markersize=1, latlon=True)
    map.plot(lonsSouth, latsSouth, 'o', color='darkviolet', markeredgecolor='none', markersize=1, latlon=True)
    
    map.drawcoastlines(linewidth=0.1)
    map.fillcontinents(color='#222222', lake_color='#222222')
    map.drawparallels(np.arange(10,60,5), linewidth=.2, labels=[1,1,0,0], fontsize=8)
    map.drawmeridians([-174, -170, -166, -162, -158, -154, -150, -146, -142], linewidth=.2, labels=[0,0,0,1], fontsize=8)
    map.shadedrelief()
    return



def kuroshio():
    ################ meridional elongation ######################

    h, w = 80, 200
    #cent = 5045   ## near 158W
    cent = 4495   ## near 180
    lonsPolar, latsPolar = tracerBlock(X, Y, cent-w, cent+w, 3300, 3400)    
    lonsNorth, latsNorth = tracerBlock(X, Y, cent-w, cent+w, 3200, 3300)    
    #lonsTZ, latsTZ = tracerBlock(X, Y, 4832-w, 4832+w, 3025, 3025+h)
    lonsTZ1, latsTZ1 = tracerBlock(X, Y, cent-w, cent+w, 2990, 3030)
    lonsTZ2, latsTZ2 = tracerBlock(X, Y, cent-w, cent+w, 3030, 3060)
    lonsTZ3, latsTZ3 = tracerBlock(X, Y, cent-w, cent+w, 3060, 3100)
    lonsTZ4, latsTZ4 = tracerBlock(X, Y, cent-w, cent+w, 3100, 3200)
    lonsSouth, latsSouth = tracerBlock(X, Y, cent-w, cent+w, 2840, 2840+h)
    ########################################################
    return lonsPolar, latsPolar, lonsNorth, latsNorth, lonsTZ1, latsTZ1, lonsTZ2, latsTZ2, lonsTZ3, latsTZ3, lonsTZ4, latsTZ4, lonsSouth, latsSouth


def NPDomain():
    #lonsPolar, latsPolar, lonsNorth, latsNorth, lonsTZ1, latsTZ1, lonsTZ2, latsTZ2, lonsTZ3, latsTZ3, lonsTZ4, latsTZ4, lonsSouth, latsSouth = bwFat()
    #lonsPolar, latsPolar, lonsNorth, latsNorth, lonsTZ1, latsTZ1, lonsTZ2, latsTZ2, lonsTZ3, latsTZ3, lonsTZ4, latsTZ4, lonsSouth, latsSouth = bwSkinny()
    lonsPolar, latsPolar, lonsNorth, latsNorth, lonsTZ1, latsTZ1, lonsTZ2, latsTZ2, lonsTZ3, latsTZ3, lonsTZ4, latsTZ4, lonsSouth, latsSouth = kuroshio()
    
    map = Basemap(llcrnrlon=-210,llcrnrlat=10.,urcrnrlon=-130,urcrnrlat=55,\
        rsphere=(6378137.00,6356752.3142),\
        resolution='l',area_thresh=1000.,projection='lcc',\
        lat_1=40.,lon_0=-175.)
    map.plot(lonsPolar, latsPolar, 'o', color='maroon', markeredgecolor='none', markersize=1, latlon=True)
    map.plot(lonsNorth, latsNorth, 'o', color='springgreen', markeredgecolor='none', markersize=1, latlon=True)
    map.plot(lonsTZ1, latsTZ1, 'o', color='crimson', markeredgecolor='none', markersize=1, latlon=True)
    map.plot(lonsTZ2, latsTZ2, 'o', color='white', markeredgecolor='none', markersize=1, latlon=True)
    map.plot(lonsTZ3, latsTZ3, 'o', color='darkslategrey', markeredgecolor='none', markersize=1, latlon=True)
    map.plot(lonsTZ4, latsTZ4, 'o', color='darkorange', markeredgecolor='none', markersize=1, latlon=True)
    map.plot(lonsSouth, latsSouth, 'o', color='darkviolet', markeredgecolor='none', markersize=1, latlon=True)
    
    map.drawcoastlines(linewidth=0.1)
    map.fillcontinents(color='#222222', lake_color='#222222')
    map.drawparallels(np.arange(10,60,5), linewidth=.2, labels=[1,1,0,0], fontsize=8)
    #map.drawmeridians([-174, -170, -166, -162, -158, -154, -150, -146, -142], linewidth=.2, labels=[0,0,0,1], fontsize=8)
    map.drawmeridians([-158, -140, 140, 160, 180], linewidth=.2, labels=[0,0,0,1], fontsize=8)
    map.shadedrelief()
    return

def site_indices(lon, lat, site_lon, site_lat):
    lon_index = np.argmin(np.abs(np.abs(lon)-site_lon))
    lat_index = np.argmin(np.abs(lat-site_lat))
    return lon_index, lat_index

def all_site_indices(lon, lat, site_lons, site_lats):
    lon_index = []
    lat_index = []
    for ll in site_lons:
        lon_index.append(np.argmin(np.abs(np.abs(lon)-ll)))
    for ll in site_lats:
        lat_index.append(np.argmin(np.abs(lat-ll)))
    return lon_index, lat_index

In [3]:
# load observational data points

infile ='../data/obs_coords.xlsx'
obs = pd.read_excel(infile)

print obs.columns

# import MDT data
infile = '../../kuroshio_chapter/data/sla/dt_global_allsat_msla_h_20091020_20140106.nc'
mdt_data = Dataset(infile, mode='r')

mdt_data.variables

mdt = np.squeeze(mdt_data.variables['sla'])
lats = mdt_data.variables['lat'][:]
lons = mdt_data.variables['lon'][:]

lon_mdt, lat_mdt = np.meshgrid(lons, lats)

mdt[mdt<-3] = np.nan

Index([u'year', u'month', u'ref', u'lat', u'lon', u'method', u'result'], dtype='object')


In [ ]:
date = ['Oct 21', 'Oct 22']

hw = 2

fig0, [ax1, ax2] = plt.subplots(1, 2, sharey=True, figsize=(9,9), dpi = 300)

maxday = 7
palette = plt.get_cmap('RdYlBu_r', (maxday-1)*2)
norm = colors.Normalize(vmin=1, vmax=maxday)


for da in date:

    if da == 'Oct 18':
        #Oct 18
        indir = '../data/trajectories/Oct18/'
        site_lons=[143.5, 143.5, 143.5, 143.5, 143.5, 143.5, 143.5, 143.5]
        site_lats=[36.6102, 35.9347, 35.8443, 35.7645, 35.6807, 35.5985, 35.5208, 35.4323]
    elif da == 'Oct 19':
        #Oct 19
        indir = '../data/trajectories/Oct19/'
        site_lons=[144., 144., 144., 144., 144., 144., 144., 144.]
        site_lats=[36.0485, 36.127, 36.2273, 36.2977, 36.3885, 36.471, 36.5502, 36.6308]
    elif da =='Oct 20':
        #Oct 20
        indir = '../data/trajectories/Oct20/'
        site_lons=[144.5, 144.5, 144.5, 144.5, 144.5, 144.5, 144.5, 144.5]
        site_lats=[36.5, 36.4148, 36.3287, 36.1602, 36.079, 35.9967, 35.9117, 35.831, 35.743]
    elif da == 'Oct 21':
        #Oct 21
        indir = '../data/trajectories/Oct21/'
        site_lons=[145., 145., 145., 145., 145., 145., 145., 145., 145.5]
        site_lats=[35.587, 35.6712, 35.7537, 35.8347, 35.916, 35.9997, 36.0818, 36.1672, 35.6667]
        jdate = '0002009294'
        aa = ax1
        
    elif da == 'Oct 22':
        #Oct 22
        indir = '../data/trajectories/Oct22/'
        site_lons=[145.5, 145.5, 145.5, 145.5, 145.5, 145.5]
        site_lats=[35.5833, 35.5, 35.4167, 35.4167, 35.4167, 35.1667]
        jdate = '0002009295'
        aa = ax2

    m = Basemap(projection='merc',llcrnrlat=30.,urcrnrlat=40.05,\
                llcrnrlon=135.,urcrnrlon=155.,lat_ts=0,resolution='c', ax = aa)

    mcon = plt.get_cmap('RdYlBu_r', 30)

    xii, yii = m(lon_mdt, lat_mdt)
    # mm = m.contourf(xii,yii,mdt, cmap = mcon, vmin = -0.5, vmax = 1.75, lw = 3, zorder = 0, alpha = 0.5)
    # m.colorbar(mm)

    m.contour(xii,yii,mdt,levels = np.arange(-2, 2, 0.1), colors = 'grey', lw = 5, zorder = 0, alpha = 0.5)

    m.drawcoastlines()
    m.drawmapboundary(fill_color='white')
    m.fillcontinents(color='grey',lake_color='white')

    sx, sy = m(site_lons, site_lats)
    m.plot(sx, sy,'xk', markersize = 5)

    # step through days to find trajectories
    for d in range(1,maxday + 1):
        path = '%s%s_%010d.mat' % (indir, jdate, d)
        X, Y, lon, lat = loadLCS(path)

        # set the domain
        min_lon=np.min(lon)
        max_lon=np.max(lon)
        min_lat=np.min(lat)
        max_lat=np.max(lat)
        lon = np.arange(min_lon, max_lon+0.01,0.01)
        lat = np.arange(min_lat, max_lat+0.01,0.01)

        # find domain indices for site coordinates
        lon_index, lat_index = all_site_indices(lon, lat, site_lons, site_lats)

        lons, lats = tracerBlock(X, Y, lon_index, lat_index, hw)
        x, y = m(lons, lats)

        if d < 2: 
            cb = m.scatter(x,y, s=5, c=np.ones(len(x))*d, vmin = 1, vmax = maxday, cmap = palette, zorder = 2)
        else:
            m.scatter(x,y, s=5, c=np.ones(len(x))*d, vmin = 1, vmax = maxday, cmap = palette, alpha = 0.75,zorder = 2)
            for dd in range(len(xold)):
                m.plot([xold[dd],x[dd]], [yold[dd], y[dd]], '-', color = palette(norm(d-0.5)), alpha = 0.5, zorder = 1)
        xold = x
        yold = y
        print d
    
m.drawparallels(np.arange(-50,80,5),labels=[1,0,0,0])
m.drawmeridians(np.arange(-180,180,10),labels=[0,0,0,1])
m.colorbar(cb)
cb.set_label('days before reaching sample location')

plt.tick_params(axis='both', which='major')
plt.tick_params(axis='both', which='minor')
plt.xlabel('Longitude', labelpad = 30)
plt.ylabel('Latitude', labelpad = 50)

plt.show()

fig0.savefig('../figures/test_traj.png', dpi = 300)

1
2
3